# Meda - List
## Info
Gleiche Firma wie BGP. ABER: In diesem PDF gibt es Einträge ohne Total. Daher wurde die Logik, leere Zeilen zu entfernen, geändert

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
#import consts

## Import PDF

In [2]:
# Read pdf into DataFrame
df = tabula.read_pdf("Report Meda 2019.pdf", pages='all', lattice=True)

## Format Table

In [6]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']

#Remove Strings
df_export.loc[df_export['empty0'] == 'H\rC\rP\rs', 'empty0'] = np.nan
df_export.loc[df_export['empty0'] == 'H\rC\rO\rs', 'empty0'] = np.nan
df_export.loc[df_export['empty0'] == 'H', 'empty0'] = np.nan

#Shift rows
df_export[df_export['empty0'].notna()] = df_export[df_export['empty0'].notna()].shift(1, axis='columns')

#Remove rows which have no values
df_export = df_export.dropna(subset=['sponsorship', 'total'], how='all')
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.reset_index(drop=True)

#Remove \r
df_export = remove_carination(df_export, " ")

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = remove_dots(df_export)
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Remove Titles in Name
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Remove empty
df_export.drop(columns=['empty0', 'empty1'], inplace=True)

#Set Type
df_export = set_type_by_alphabetical_order(df_export)

#shift Surname/Lastname
df_export = revert_name(df_export, ' ')

#basic string conversion
df_export = basic_string_conversion(df_export)

#Add PLZ
df_export = add_plz(df_export)

#Export
export_list(df_export, 'meda')

saved


In [ ]:
#write_to_excel(df_export, 'tmp.xlsx', open=True)